In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm  # import tqdm for progress bar 
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
target_labels = ['Peach___Bacterial_spot', 'Peach___healthy', 'Strawberry___healthy', 'Strawberry___Leaf_scorch']

In [3]:
main_path = 'data_penyakit/dataset/train'

In [4]:
# Load the images and labels
filename = []
X = []
y = []
for label in target_labels:
    label_dir = os.path.join(main_path, label)
    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        filename.append(img_file)
        X.append(img_array)
        y.append(target_labels.index(label))

In [5]:
from tensorflow.keras.utils import to_categorical

In [6]:
X = np.array(X)
y = to_categorical(y, num_classes=len(target_labels))

In [7]:
X.shape

(7164, 128, 128, 3)

In [8]:
y.shape

(7164, 4)

In [9]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, filename_train, filename_test = train_test_split(X, y, filename, test_size=0.2, random_state=42)

In [10]:
X_train.shape

(5731, 128, 128, 3)

In [11]:
X_test.shape

(1433, 128, 128, 3)

In [12]:
y_train.shape

(5731, 4)

In [13]:
y_test.shape

(1433, 4)

In [14]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.models import Sequential

In [15]:
len(target_labels)

4

In [16]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, kernel_size= (3,3), activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(64,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(96,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(32,kernel_size= (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size =(2,2,)))
cnn_model.add(BatchNormalization())

cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation = 'relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(target_labels), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10


2023-05-31 14:16:10.507494: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


180/180 [==============================] - 70s 380ms/step - loss: 0.2738 - accuracy: 0.9030 - val_loss: 2.1011 - val_accuracy: 0.7013
Epoch 2/10
180/180 [==============================] - 49s 271ms/step - loss: 0.1218 - accuracy: 0.9590 - val_loss: 1.0074 - val_accuracy: 0.7439
Epoch 3/10
180/180 [==============================] - 109s 605ms/step - loss: 0.0904 - accuracy: 0.9721 - val_loss: 0.2478 - val_accuracy: 0.9288
Epoch 4/10
180/180 [==============================] - 137s 761ms/step - loss: 0.1102 - accuracy: 0.9621 - val_loss: 0.3956 - val_accuracy: 0.8786
Epoch 5/10
180/180 [==============================] - 132s 735ms/step - loss: 0.0768 - accuracy: 0.9737 - val_loss: 0.2530 - val_accuracy: 0.9086
Epoch 6/10
180/180 [==============================] - 131s 729ms/step - loss: 0.0545 - accuracy: 0.9817 - val_loss: 0.1409 - val_accuracy: 0.9477
Epoch 7/10
180/180 [==============================] - 135s 751ms/step - loss: 0.0764 - accuracy: 0.9730 - val_loss: 0.1077 - val_accuracy

In [18]:
# Evaluate the performance of the trained CNN model on the test set
cnn_scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print("CNN Model Accuracy: %.2f%%" % (cnn_scores[1] * 100))

CNN Model Accuracy: 98.74%


In [19]:
cnn_scores

[0.03683667629957199, 0.9874389171600342]

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Use the model to predict the test dataset
y_pred_prob = cnn_model.predict(X_test)

# Convert probabilities to classes
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it to classes as well
y_test_classes = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_classes, y_pred)

# Calculate precision
precision = precision_score(y_test_classes, y_pred, average='weighted') 

# Calculate recall
recall = recall_score(y_test_classes, y_pred, average='weighted') 

# Calculate F1 score
f1 = f1_score(y_test_classes, y_pred, average='weighted')

# Print the metrics
print("Accuracy: %.2f%%" % (accuracy * 100))
print("Precision: %.2f%%" % (precision * 100))
print("Recall: %.2f%%" % (recall * 100))
print("F1 score: %.2f%%" % (f1 * 100))

45/45 [==============================] - 10s 202ms/step
Accuracy: 98.74%
Precision: 98.75%
Recall: 98.74%
F1 score: 98.74%


In [21]:
from tabulate import tabulate

# Create a DataFrame with true labels, predicted labels, and target labels
df = pd.DataFrame({'Filename': filename_test,
                   'True Labels': [target_labels[label] for label in y_test_classes],
                   'Predicted Labels': [target_labels[label] for label in y_pred]})

# Convert DataFrame to tabular format
table = tabulate(df, headers='keys', tablefmt='psql')

# Print the table
print(table)

# Save the DataFrame to a CSV file
df.to_csv('labels_predictions-cnn-peach+strawberry.csv', index=False)

+------+-------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                | True Labels              | Predicted Labels         |
|------+-------------------------------------------------------------------------+--------------------------+--------------------------|
|    0 | 934ec31d-5e5f-453d-a967-ab5debfd1d09___Rutg._Bact.S 2173.JPG            | Peach___Bacterial_spot   | Peach___Bacterial_spot   |
|    1 | 5796cd59-cde6-4b01-bfd0-5b6c534fabee___Rutg._Bact.S 2312.JPG            | Peach___Bacterial_spot   | Peach___Bacterial_spot   |
|    2 | 72098b1a-ef4a-47eb-bb1f-f23b126b7057___RS_L.Scorch 1324_flipLR.JPG      | Strawberry___Leaf_scorch | Strawberry___Leaf_scorch |
|    3 | 84b41aa3-4e4f-42a7-890b-db2892047331___RS_L.Scorch 1562.JPG             | Strawberry___Leaf_scorch | Strawberry___Leaf_scorch |
|    4 | e4f0a3a5-2de6-4496-83fa-8a3c015e

In [22]:
# Filter rows where true label and predicted label do not match
false_predictions = df[df['True Labels'] != df['Predicted Labels']]

# Convert DataFrame to tabular format
table = tabulate(false_predictions, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table)

# Save the false predictions to a CSV file
false_predictions.to_csv('false_predictions-cnn-peach+strawberry.csv', index=False)

False Predictions:
+------+--------------------------------------------------------------------+--------------------------+------------------------+
|      | Filename                                                           | True Labels              | Predicted Labels       |
|------+--------------------------------------------------------------------+--------------------------+------------------------|
|   18 | c4cf1b6a-7d05-4064-86ec-6224afb6d936___Rut._Bact.S 0914.JPG        | Peach___Bacterial_spot   | Peach___healthy        |
|   53 | 8ff4c576-976d-4488-83a5-cc5a694028f5___Rutg._HL 3566.JPG           | Peach___healthy          | Peach___Bacterial_spot |
|   61 | d94dce48-f8ec-40fa-9ea7-a0ad05deceb2___Rutg._Bact.S 1344.JPG       | Peach___Bacterial_spot   | Peach___healthy        |
|   84 | 1ef09571-8a75-4c3e-8275-321d0d670509___Rutg._HL 2465_180deg.JPG    | Peach___healthy          | Peach___Bacterial_spot |
|  192 | 06b67d4d-4bc8-4f4e-baa1-2477addaf354___Rut._Bact.S 3469.JPG   

In [23]:
# Extract the features using the trained CNN model
cnn_features = cnn_model.predict(X_train)

180/180 [==============================] - 35s 194ms/step


# rf

In [24]:
import time

In [25]:
# import the Random Forest classifier
from RF_Manual import RandomForest

In [26]:
# Train a Random Forest classifier on the extracted features
start_time = time.time()
rf_classifier = RandomForest(n_trees=100, max_depth=42)
rf_classifier.fit(cnn_features, np.argmax(y_train, axis=1))

# count time for training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

Training time: 92.10398888587952 seconds


In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the performance of the Random Forest classifier on the test set
rf_features = cnn_model.predict(X_test)
rf_features = rf_features.reshape(rf_features.shape[0], -1)
rf_predictions = rf_classifier.predict(rf_features)
rf_accuracy = np.mean(rf_predictions == np.argmax(y_test, axis=1))
print("Random Forest Classifier Accuracy: %.2f%%" % (rf_accuracy * 100))

# Calculate and print the Precision
rf_precision = precision_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Precision: %.2f" % rf_precision)

# Calculate and print the Recall
rf_recall = recall_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier Recall: %.2f" % rf_recall)

# Calculate and print the F1 score
rf_f1_score = f1_score(np.argmax(y_test, axis=1), rf_predictions, average='weighted')
print("Random Forest Classifier F1 Score: %.2f" % rf_f1_score)

45/45 [==============================] - 7s 156ms/step
Random Forest Classifier Accuracy: 99.23%
Random Forest Classifier Precision: 0.99
Random Forest Classifier Recall: 0.99
Random Forest Classifier F1 Score: 0.99


In [28]:
# Create a DataFrame with true labels and predicted labels
df_rf = pd.DataFrame({'Filename': filename_test,
                      'True Labels': [target_labels[label] for label in y_test_classes],
                      'Predicted Labels': [target_labels[label] for label in rf_predictions]})

# Convert DataFrame to tabular format
table_rf = tabulate(df_rf, headers='keys', tablefmt='psql')

# Print the table
print(table_rf)

# Save the DataFrame to a CSV file
df_rf.to_csv('labels_predictions-rf-peach+strawberry.csv', index=False)

+------+-------------------------------------------------------------------------+--------------------------+--------------------------+
|      | Filename                                                                | True Labels              | Predicted Labels         |
|------+-------------------------------------------------------------------------+--------------------------+--------------------------|
|    0 | 934ec31d-5e5f-453d-a967-ab5debfd1d09___Rutg._Bact.S 2173.JPG            | Peach___Bacterial_spot   | Peach___Bacterial_spot   |
|    1 | 5796cd59-cde6-4b01-bfd0-5b6c534fabee___Rutg._Bact.S 2312.JPG            | Peach___Bacterial_spot   | Peach___Bacterial_spot   |
|    2 | 72098b1a-ef4a-47eb-bb1f-f23b126b7057___RS_L.Scorch 1324_flipLR.JPG      | Strawberry___Leaf_scorch | Strawberry___Leaf_scorch |
|    3 | 84b41aa3-4e4f-42a7-890b-db2892047331___RS_L.Scorch 1562.JPG             | Strawberry___Leaf_scorch | Strawberry___Leaf_scorch |
|    4 | e4f0a3a5-2de6-4496-83fa-8a3c015e

In [29]:
# Filter rows where true label and predicted label do not match
false_predictions_rf = df_rf[df_rf['True Labels'] != df_rf['Predicted Labels']]

# Convert DataFrame to tabular format
table_rf = tabulate(false_predictions_rf, headers='keys', tablefmt='psql')

# Print the false predictions table
print("False Predictions:")
print(table_rf)

# Save the false predictions to a CSV file
false_predictions_rf.to_csv('false_predictions-rf-peach+strawberry.csv', index=False)

False Predictions:
+------+-----------------------------------------------------------------+------------------------+--------------------------+
|      | Filename                                                        | True Labels            | Predicted Labels         |
|------+-----------------------------------------------------------------+------------------------+--------------------------|
|   18 | c4cf1b6a-7d05-4064-86ec-6224afb6d936___Rut._Bact.S 0914.JPG     | Peach___Bacterial_spot | Peach___healthy          |
|   53 | 8ff4c576-976d-4488-83a5-cc5a694028f5___Rutg._HL 3566.JPG        | Peach___healthy        | Peach___Bacterial_spot   |
|   61 | d94dce48-f8ec-40fa-9ea7-a0ad05deceb2___Rutg._Bact.S 1344.JPG    | Peach___Bacterial_spot | Peach___healthy          |
|   84 | 1ef09571-8a75-4c3e-8275-321d0d670509___Rutg._HL 2465_180deg.JPG | Peach___healthy        | Peach___Bacterial_spot   |
|  192 | 06b67d4d-4bc8-4f4e-baa1-2477addaf354___Rut._Bact.S 3469.JPG     | Peach___Bacterial